In [19]:
with open("input.txt", "r") as file:
    content = file.readlines()


In [20]:
def str_clean(sent):

    append_to_prev = False

    if len(sent) > 0 and not sent[0] =='[':
        append_to_prev = True
        sent = sent[:-1]
        return append_to_prev, sent

    else:
        sent = sent.split(":")[2:]

        if len(sent) == 0 or sent[-1].isspace():
            return None, None
        
        sent[-1] = sent[-1][:-1]
        sent = ":".join(sent)
        return append_to_prev, sent


content_clean = []
clean_index = []
for i, c in enumerate(content):
    append_to_prev, sent = str_clean(c)
    if sent is not None:
        if append_to_prev:
            content_clean[-1] += ' \n' + sent
        else:
            content_clean.append(sent)
            clean_index.append(i)



In [ ]:
# Test block

In [34]:
import re
import requests

from transformers import pipeline
# from transformers import AutoTokenizer, AutoModelForTokenClassification



### For ner
# tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
# model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
# ner_nlp = pipeline("ner", model=model, tokenizer=tokenizer)
ner_nlp = pipeline("ner")



### For query_llm 
# (login to HF using 'huggingface-cli login --token $HUGGINGFACE_TOKEN' before this)
# model_llm = "meta-llama/Llama-2-7b-chat-hf"
# tokenizer = AutoTokenizer.from_pretrained(model_llm)
# chat_llm = pipeline(
#     "text-generation",
#     model=model_llm,
#    # torch_dtype=torch.float16,
#     device_map="auto",
# )
chat_llm = pipeline("text-generation")



### For chain_of_density_summarise (we're using the regular summariser for now)
summarizer = pipeline("summarization")




def get_context_from_links(text):
    context = ''
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in urls:
        f = requests.get(url)
        context += '\n {}'.format(f.text)
    return chain_of_density_summarise(context, '')



def chain_of_density_summarise(prev, next):
    # Regular summariser for now
    summarized = summarizer(prev + next, min_length=0, max_length=300)
    return summarized



def query_llm(prompt):
    sequences = chat_llm(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        # eos_token_id=tokenizer.eos_token_id,
        max_length=200,
    )
    
    return sequences[0]['generated_text']

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
No model was supplied, defaulted

In [36]:
summaries = []

print('total entries: {}'.format(len(content_clean)))

# for i in range(len(content_clean)):
for i in range(0):

    print('working on entry {}'.format(i))

    ner_content = ner_nlp(content_clean[i])
    overall_context = content_clean[i]

    for d in ner_content:
        prompt = "What is {} ? \nContext: {}. Keep it very brief.".format(d['word'], content_clean[i])
        response = query_llm(prompt)
        overall_context += '\n' + response

    context_from_links = get_context_from_links(content_clean[i])
    overall_context += "\n\n" + context_from_links
        

    added = False
    for s in summaries:
        prompt = "Are these things part of the same conversation ? {} and {}. Strictly reply in 'yes' or 'no'.".format(s['content'], overall_context)
        response = query_llm(prompt)

        if 'yes' in response:
            s['content'] = chain_of_density_summarise(s['content'], overall_context)
            s['index'].append(clean_index[i])
            added = True
        else:
            pass

    if added is False:
        s_new = {
            'content': chain_of_density_summarise('', overall_context),
            'index': [clean_index[i]]
        }
        summaries.append(s_new)


total entries: 254


In [39]:
### Testing


def get_context_from_links2(text):
    context = ''
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in urls:
        f = requests.get(url)
        context += '\n {}'.format(f.text)
        print(url)
        print(f.text)
    
    print("\n\n")
    print(context)
    # return chain_of_density_summarise(context, '')



i = 1

print('working on entry {}'.format(i))

ner_content = ner_nlp(content_clean[i])
overall_context = content_clean[i]

for d in ner_content:
    prompt = "What is {} ? \nContext: {}. Keep it very brief.".format(d['word'], content_clean[i])
    response = query_llm(prompt)
    overall_context += '\n' + response

context_from_links = get_context_from_links2(content_clean[i])
overall_context += "\n\n" + context_from_links

working on entry 1
http://ai.google.dev
<!doctype html>
<html 
      lang="en"
      dir="ltr">
  <head>
    <meta name="google-signin-client-id" content="157101835696-ooapojlodmuabs2do2vuhhnf90bccmoi.apps.googleusercontent.com">
    <meta name="google-signin-scope"
          content="profile email https://www.googleapis.com/auth/developerprofiles https://www.googleapis.com/auth/developerprofiles.award">
    <meta property="og:site_name" content="Google AI for Developers">
    <meta property="og:type" content="website"><meta name="theme-color" content="#1967d2"><meta charset="utf-8">
    <meta content="IE=Edge" http-equiv="X-UA-Compatible">
    <meta name="viewport" content="width=device-width, initial-scale=1">
    

    <link rel="manifest" href="/_pwa/googledevai/manifest.json"
          crossorigin="use-credentials">
    <link rel="preconnect" href="//www.gstatic.com" crossorigin>
    <link rel="preconnect" href="//fonts.gstatic.com" crossorigin>
    <link rel="preconnect" href="//

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:

added = False
for s in summaries:
    prompt = "Are these things part of the same conversation ? {} and {}. Strictly reply in 'yes' or 'no'.".format(s['content'], overall_context)
    response = query_llm(prompt)

    if 'yes' in response:
        s['content'] = chain_of_density_summarise(s['content'], overall_context)
        s['index'].append(clean_index[i])
        added = True
    else:
        pass

if added is False:
    s_new = {
        'content': chain_of_density_summarise('', overall_context),
        'index': [clean_index[i]]
    }
    summaries.append(s_new)